In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
from scipy.spatial import cKDTree
from math import *
np.random.seed(123)
import urllib.request
import urllib, os

In [2]:
import pandas as pd
import xarray as xr
import math
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [3]:
import numpy as np
import xarray as xr
import xesmf as xe

In [4]:
df_prep = xr.open_dataset(r'/home/shubham/ugp/cleaned/Precipitation_1979-2022(ver2).nc')

In [5]:
df_prep

<xarray.Dataset>
Dimensions:        (time: 517, y: 72, x: 144)
Coordinates:
    lon            (x) float32 ...
    lat            (y) float32 ...
  * time           (time) datetime64[ns] 1979-02-01 1979-03-01 ... 2022-02-01
Dimensions without coordinates: y, x
Data variables:
    precipitation  (time, y, x) float32 ...
Attributes:
    description:  Precipitation(in mm/day) data NOAA NCEP CPC Merged_Analysis...

In [6]:
df_prep["slice"] = df_prep["precipitation"].isel(time=0)
df_prep

<xarray.Dataset>
Dimensions:        (time: 517, y: 72, x: 144)
Coordinates:
    lon            (x) float32 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
    lat            (y) float32 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
  * time           (time) datetime64[ns] 1979-02-01 1979-03-01 ... 2022-02-01
Dimensions without coordinates: y, x
Data variables:
    precipitation  (time, y, x) float32 ...
    slice          (y, x) float32 ...
Attributes:
    description:  Precipitation(in mm/day) data NOAA NCEP CPC Merged_Analysis...

In [7]:
df_prep_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90,90, 2.0)),
        "lon": (["lon"], np.arange(0, 360, 2)),
    }
)

regridder = xe.Regridder(df_prep, df_prep_out, 'bilinear')
regridder

Create weight file: bilinear_72x144_90x180.nc


xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_72x144_90x180.nc 
Reuse pre-computed weights? False 
Input grid shape:           (72, 144) 
Output grid shape:          (90, 180) 
Output grid dimension name: ('lat', 'lon') 
Periodic in longitude?      False

In [8]:
df_out = regridder(df_prep)
df_out

using dimensions ('y', 'x') from data variable precipitation as the horizontal dimensions for this dataset.


/home/shubham/anaconda3/envs/ugp_env/lib/python3.7/site-packages/xesmf/frontend.py:486: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


<xarray.Dataset>
Dimensions:        (time: 517, lat: 90, lon: 180)
Coordinates:
  * time           (time) datetime64[ns] 1979-02-01 1979-03-01 ... 2022-02-01
  * lon            (lon) int64 0 2 4 6 8 10 12 ... 346 348 350 352 354 356 358
  * lat            (lat) float64 -90.0 -88.0 -86.0 -84.0 ... 82.0 84.0 86.0 88.0
Data variables:
    precipitation  (time, lat, lon) float64 0.0 0.0 0.0 ... 0.4532 0.4468 0.4453
    slice          (lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.196 0.188 0.187
Attributes:
    regrid_method:  bilinear

In [9]:
df_out = df_out.drop(['slice'])

In [10]:
df_out['time'][278:-3]

<xarray.DataArray 'time' (time: 236)>
array(['2002-04-01T00:00:00.000000000', '2002-05-01T00:00:00.000000000',
       '2002-06-01T00:00:00.000000000', ..., '2021-09-01T00:00:00.000000000',
       '2021-10-01T00:00:00.000000000', '2021-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01

In [11]:
df_out = df_out['precipitation'][278:-3][:][:]

In [12]:
df_out

<xarray.DataArray 'precipitation' (time: 236, lat: 90, lon: 180)>
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 3.44071380e-01, 3.41984768e-01, ...,
         3.63688424e-01, 3.61287802e-01, 3.54672087e-01],
        [0.00000000e+00, 2.47334960e-01, 2.40917370e-01, ...,
         2.90918735e-01, 2.82117967e-01, 2.60740760e-01],
        ...,
        [0.00000000e+00, 3.04919487e-01, 3.31804869e-01, ...,
         2.74793301e-01, 2.81195831e-01, 2.84809867e-01],
        [0.00000000e+00, 2.72115533e-01, 3.14204017e-01, ...,
         3.05193759e-01, 2.98796683e-01, 2.67208694e-01],
        [0.00000000e+00, 2.20413162e-01, 2.32910731e-01, ...,
         2.93601540e-01, 2.74404161e-01, 2.41907298e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.81013051e-01, 1.80406331e-01, ...,
         1.89409182e-01, 1.84607935e-01, 1.81913690e-01],
        [0.00000000e+00, 2.32608944e-01, 2.32206293e-01, ...,
         2.62106099e-01, 2.46908245e-01, 2.37314991e-01],
...
        [0.00000000e+00, 7.55563268e-01, 8.09968832e-01, ...,
         6.56063692e-01, 6.64063227e-01, 6.86157963e-01],
        [0.00000000e+00, 4.70723045e-01, 5.13160572e-01, ...,
         3.76056065e-01, 3.84055600e-01, 4.06118815e-01],
        [0.00000000e+00, 2.84540487e-01, 2.97524832e-01, ...,
         2.56311581e-01, 2.58712203e-01, 2.65327914e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.77110541e-01, 2.74304687e-01, ...,
         2.87305654e-01, 2.81706724e-01, 2.78911820e-01],
        [0.00000000e+00, 3.18304201e-01, 3.11903011e-01, ...,
         3.30000013e-01, 3.30000013e-01, 3.23702633e-01],
        ...,
        [0.00000000e+00, 2.99856305e-01, 3.28728592e-01, ...,
         2.34911263e-01, 2.42112306e-01, 2.61940557e-01],
        [0.00000000e+00, 2.02241801e-01, 2.06324703e-01, ...,
         1.86109760e-01, 1.86910991e-01, 1.89131027e-01],
        [0.00000000e+00, 1.20018649e-01, 1.19609615e-01, ...,
         1.24007615e-01, 1.24007615e-01, 1.24014921e-01]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [13]:
pd.to_datetime(df_out['time'])[0]

Timestamp('2002-04-01 00:00:00')

In [14]:
new_index = pd.period_range(pd.to_datetime(df_out['time'])[0],pd.to_datetime(df_out['time'])[-1],freq='M')

In [15]:
new_index = pd.DataFrame(new_index)
new_index

,0
0,2002-04
1,2002-05
2,2002-06
3,2002-07
4,2002-08
...,...
231,2021-07
232,2021-08
233,2021-09
234,2021-10


In [16]:
df_out

<xarray.DataArray 'precipitation' (time: 236, lat: 90, lon: 180)>
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 3.44071380e-01, 3.41984768e-01, ...,
         3.63688424e-01, 3.61287802e-01, 3.54672087e-01],
        [0.00000000e+00, 2.47334960e-01, 2.40917370e-01, ...,
         2.90918735e-01, 2.82117967e-01, 2.60740760e-01],
        ...,
        [0.00000000e+00, 3.04919487e-01, 3.31804869e-01, ...,
         2.74793301e-01, 2.81195831e-01, 2.84809867e-01],
        [0.00000000e+00, 2.72115533e-01, 3.14204017e-01, ...,
         3.05193759e-01, 2.98796683e-01, 2.67208694e-01],
        [0.00000000e+00, 2.20413162e-01, 2.32910731e-01, ...,
         2.93601540e-01, 2.74404161e-01, 2.41907298e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.81013051e-01, 1.80406331e-01, ...,
         1.89409182e-01, 1.84607935e-01, 1.81913690e-01],
        [0.00000000e+00, 2.32608944e-01, 2.32206293e-01, ...,
         2.62106099e-01, 2.46908245e-01, 2.37314991e-01],
...
        [0.00000000e+00, 7.55563268e-01, 8.09968832e-01, ...,
         6.56063692e-01, 6.64063227e-01, 6.86157963e-01],
        [0.00000000e+00, 4.70723045e-01, 5.13160572e-01, ...,
         3.76056065e-01, 3.84055600e-01, 4.06118815e-01],
        [0.00000000e+00, 2.84540487e-01, 2.97524832e-01, ...,
         2.56311581e-01, 2.58712203e-01, 2.65327914e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.77110541e-01, 2.74304687e-01, ...,
         2.87305654e-01, 2.81706724e-01, 2.78911820e-01],
        [0.00000000e+00, 3.18304201e-01, 3.11903011e-01, ...,
         3.30000013e-01, 3.30000013e-01, 3.23702633e-01],
        ...,
        [0.00000000e+00, 2.99856305e-01, 3.28728592e-01, ...,
         2.34911263e-01, 2.42112306e-01, 2.61940557e-01],
        [0.00000000e+00, 2.02241801e-01, 2.06324703e-01, ...,
         1.86109760e-01, 1.86910991e-01, 1.89131027e-01],
        [0.00000000e+00, 1.20018649e-01, 1.19609615e-01, ...,
         1.24007615e-01, 1.24007615e-01, 1.24014921e-01]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [17]:
type(new_index[0][0])

pandas._libs.tslibs.period.Period

In [18]:
new_index = new_index[0].dt.to_timestamp()
type(new_index)

pandas.core.series.Series

In [19]:
new_index

0     2002-04-01
1     2002-05-01
2     2002-06-01
3     2002-07-01
4     2002-08-01
         ...    
231   2021-07-01
232   2021-08-01
233   2021-09-01
234   2021-10-01
235   2021-11-01
Name: 0, Length: 236, dtype: datetime64[ns]

In [20]:
df_out['time'] = pd.to_datetime(df_out['time']).strftime("%Y-%m")

In [21]:
df_out['time'] = pd.to_datetime(df_out['time'])
df_out

<xarray.DataArray 'precipitation' (time: 236, lat: 90, lon: 180)>
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 3.44071380e-01, 3.41984768e-01, ...,
         3.63688424e-01, 3.61287802e-01, 3.54672087e-01],
        [0.00000000e+00, 2.47334960e-01, 2.40917370e-01, ...,
         2.90918735e-01, 2.82117967e-01, 2.60740760e-01],
        ...,
        [0.00000000e+00, 3.04919487e-01, 3.31804869e-01, ...,
         2.74793301e-01, 2.81195831e-01, 2.84809867e-01],
        [0.00000000e+00, 2.72115533e-01, 3.14204017e-01, ...,
         3.05193759e-01, 2.98796683e-01, 2.67208694e-01],
        [0.00000000e+00, 2.20413162e-01, 2.32910731e-01, ...,
         2.93601540e-01, 2.74404161e-01, 2.41907298e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.81013051e-01, 1.80406331e-01, ...,
         1.89409182e-01, 1.84607935e-01, 1.81913690e-01],
        [0.00000000e+00, 2.32608944e-01, 2.32206293e-01, ...,
         2.62106099e-01, 2.46908245e-01, 2.37314991e-01],
...
        [0.00000000e+00, 7.55563268e-01, 8.09968832e-01, ...,
         6.56063692e-01, 6.64063227e-01, 6.86157963e-01],
        [0.00000000e+00, 4.70723045e-01, 5.13160572e-01, ...,
         3.76056065e-01, 3.84055600e-01, 4.06118815e-01],
        [0.00000000e+00, 2.84540487e-01, 2.97524832e-01, ...,
         2.56311581e-01, 2.58712203e-01, 2.65327914e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.77110541e-01, 2.74304687e-01, ...,
         2.87305654e-01, 2.81706724e-01, 2.78911820e-01],
        [0.00000000e+00, 3.18304201e-01, 3.11903011e-01, ...,
         3.30000013e-01, 3.30000013e-01, 3.23702633e-01],
        ...,
        [0.00000000e+00, 2.99856305e-01, 3.28728592e-01, ...,
         2.34911263e-01, 2.42112306e-01, 2.61940557e-01],
        [0.00000000e+00, 2.02241801e-01, 2.06324703e-01, ...,
         1.86109760e-01, 1.86910991e-01, 1.89131027e-01],
        [0.00000000e+00, 1.20018649e-01, 1.19609615e-01, ...,
         1.24007615e-01, 1.24007615e-01, 1.24014921e-01]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [22]:
df_out['time'][:]

<xarray.DataArray 'time' (time: 236)>
array(['2002-04-01T00:00:00.000000000', '2002-05-01T00:00:00.000000000',
       '2002-06-01T00:00:00.000000000', ..., '2021-09-01T00:00:00.000000000',
       '2021-10-01T00:00:00.000000000', '2021-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01

In [23]:
new_index

0     2002-04-01
1     2002-05-01
2     2002-06-01
3     2002-07-01
4     2002-08-01
         ...    
231   2021-07-01
232   2021-08-01
233   2021-09-01
234   2021-10-01
235   2021-11-01
Name: 0, Length: 236, dtype: datetime64[ns]

In [24]:
df_out['lat']

<xarray.DataArray 'lat' (lat: 90)>
array([-90., -88., -86., -84., -82., -80., -78., -76., -74., -72., -70., -68.,
       -66., -64., -62., -60., -58., -56., -54., -52., -50., -48., -46., -44.,
       -42., -40., -38., -36., -34., -32., -30., -28., -26., -24., -22., -20.,
       -18., -16., -14., -12., -10.,  -8.,  -6.,  -4.,  -2.,   0.,   2.,   4.,
         6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,  22.,  24.,  26.,  28.,
        30.,  32.,  34.,  36.,  38.,  40.,  42.,  44.,  46.,  48.,  50.,  52.,
        54.,  56.,  58.,  60.,  62.,  64.,  66.,  68.,  70.,  72.,  74.,  76.,
        78.,  80.,  82.,  84.,  86.,  88.])
Coordinates:
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [25]:
df_out

<xarray.DataArray 'precipitation' (time: 236, lat: 90, lon: 180)>
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 3.44071380e-01, 3.41984768e-01, ...,
         3.63688424e-01, 3.61287802e-01, 3.54672087e-01],
        [0.00000000e+00, 2.47334960e-01, 2.40917370e-01, ...,
         2.90918735e-01, 2.82117967e-01, 2.60740760e-01],
        ...,
        [0.00000000e+00, 3.04919487e-01, 3.31804869e-01, ...,
         2.74793301e-01, 2.81195831e-01, 2.84809867e-01],
        [0.00000000e+00, 2.72115533e-01, 3.14204017e-01, ...,
         3.05193759e-01, 2.98796683e-01, 2.67208694e-01],
        [0.00000000e+00, 2.20413162e-01, 2.32910731e-01, ...,
         2.93601540e-01, 2.74404161e-01, 2.41907298e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.81013051e-01, 1.80406331e-01, ...,
         1.89409182e-01, 1.84607935e-01, 1.81913690e-01],
        [0.00000000e+00, 2.32608944e-01, 2.32206293e-01, ...,
         2.62106099e-01, 2.46908245e-01, 2.37314991e-01],
...
        [0.00000000e+00, 7.55563268e-01, 8.09968832e-01, ...,
         6.56063692e-01, 6.64063227e-01, 6.86157963e-01],
        [0.00000000e+00, 4.70723045e-01, 5.13160572e-01, ...,
         3.76056065e-01, 3.84055600e-01, 4.06118815e-01],
        [0.00000000e+00, 2.84540487e-01, 2.97524832e-01, ...,
         2.56311581e-01, 2.58712203e-01, 2.65327914e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.77110541e-01, 2.74304687e-01, ...,
         2.87305654e-01, 2.81706724e-01, 2.78911820e-01],
        [0.00000000e+00, 3.18304201e-01, 3.11903011e-01, ...,
         3.30000013e-01, 3.30000013e-01, 3.23702633e-01],
        ...,
        [0.00000000e+00, 2.99856305e-01, 3.28728592e-01, ...,
         2.34911263e-01, 2.42112306e-01, 2.61940557e-01],
        [0.00000000e+00, 2.02241801e-01, 2.06324703e-01, ...,
         1.86109760e-01, 1.86910991e-01, 1.89131027e-01],
        [0.00000000e+00, 1.20018649e-01, 1.19609615e-01, ...,
         1.24007615e-01, 1.24007615e-01, 1.24014921e-01]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [26]:
x = df_out.reindex({"time": new_index})
x

<xarray.DataArray 'precipitation' (time: 236, lat: 90, lon: 180)>
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 3.44071380e-01, 3.41984768e-01, ...,
         3.63688424e-01, 3.61287802e-01, 3.54672087e-01],
        [0.00000000e+00, 2.47334960e-01, 2.40917370e-01, ...,
         2.90918735e-01, 2.82117967e-01, 2.60740760e-01],
        ...,
        [0.00000000e+00, 3.04919487e-01, 3.31804869e-01, ...,
         2.74793301e-01, 2.81195831e-01, 2.84809867e-01],
        [0.00000000e+00, 2.72115533e-01, 3.14204017e-01, ...,
         3.05193759e-01, 2.98796683e-01, 2.67208694e-01],
        [0.00000000e+00, 2.20413162e-01, 2.32910731e-01, ...,
         2.93601540e-01, 2.74404161e-01, 2.41907298e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.81013051e-01, 1.80406331e-01, ...,
         1.89409182e-01, 1.84607935e-01, 1.81913690e-01],
        [0.00000000e+00, 2.32608944e-01, 2.32206293e-01, ...,
         2.62106099e-01, 2.46908245e-01, 2.37314991e-01],
...
        [0.00000000e+00, 7.55563268e-01, 8.09968832e-01, ...,
         6.56063692e-01, 6.64063227e-01, 6.86157963e-01],
        [0.00000000e+00, 4.70723045e-01, 5.13160572e-01, ...,
         3.76056065e-01, 3.84055600e-01, 4.06118815e-01],
        [0.00000000e+00, 2.84540487e-01, 2.97524832e-01, ...,
         2.56311581e-01, 2.58712203e-01, 2.65327914e-01]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.77110541e-01, 2.74304687e-01, ...,
         2.87305654e-01, 2.81706724e-01, 2.78911820e-01],
        [0.00000000e+00, 3.18304201e-01, 3.11903011e-01, ...,
         3.30000013e-01, 3.30000013e-01, 3.23702633e-01],
        ...,
        [0.00000000e+00, 2.99856305e-01, 3.28728592e-01, ...,
         2.34911263e-01, 2.42112306e-01, 2.61940557e-01],
        [0.00000000e+00, 2.02241801e-01, 2.06324703e-01, ...,
         1.86109760e-01, 1.86910991e-01, 1.89131027e-01],
        [0.00000000e+00, 1.20018649e-01, 1.19609615e-01, ...,
         1.24007615e-01, 1.24007615e-01, 1.24014921e-01]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [27]:
x.to_netcdf('/home/shubham/ugp/cl_new/regrid_2_deg/Precipitation_04-2002_11-2021.nc')